In [29]:
import re
import networkx
from collections import deque

file = open('data/16.txt', 'r')
data = file.read()

file.close()

In [30]:
rates = {}
G = networkx.DiGraph()

for line in data.splitlines():
    rate = int(re.findall(r'\d+', line).pop())
    valve, *tunnels = re.findall(r'[A-Z]{2}', line)

    rates[valve] = rate

    for next in tunnels:
        G.add_edge(valve, next)

In [31]:
def get_next_valves(start):
    for valve in rates:
        if rates[valve] == 0:
            continue

        path = networkx.shortest_path(G, start, valve)
        
        if len(path) > 1:
            yield path[-1], len(path)


def get_pressure(valves, time):
    flow, timer, pressure = 0, 0, 0
    
    for valve, distance in valves.items():
        timer += distance
        pressure += flow * distance
        flow += rates[valve]
        
    return pressure + flow * (time - timer)

In [32]:
map = { valve: dict(get_next_valves(valve)) for valve in rates.keys() }

In [33]:
def find_routes(time):
    queue = deque([({}, 'AA')])

    while queue:
        valves, start = queue.pop()

        for valve, distance in map[start].items():
            if valve in valves:
                continue

            if sum(valves.values()) > time - distance:
                continue

            new = valves | { valve: distance }
            queue.append((new, valve))
            yield get_pressure(new, time), new.keys()

max(p for p, _ in find_routes(30))

1638

In [34]:
history = list(find_routes(26))

max_pressure = 0

for p1, v1 in history:
    for p2, v2 in history:
        pressure = p1 + p2

        if pressure > max_pressure and not v1 & v2:
            max_pressure = pressure

max_pressure

2400